In [1]:
const WORLD_WIDTH = 2560
const WORLD_HEIGHT = 1440
const TIMESTEPS = 10000

10000

In [2]:
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

Plant (generic function with 1 method)

In [3]:
function update!(self::Agent, food::Vector{Agent})
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

update! (generic function with 1 method)

In [7]:
function main(; show=true)
    kwargs = (
        world_width = WORLD_WIDTH,
        world_height = WORLD_HEIGHT,
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < TIMESTEPS
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    if show
        @show length(predators) length(preys) length(plants)
    end
end

main (generic function with 1 method)

In [8]:
using BenchmarkTools

In [15]:
@benchmark main(show=false)

BenchmarkTools.Trial: 12 samples with 1 evaluation.
 Range (min … max):  174.671 ms … 660.813 ms  ┊ GC (min … max): 0.90% … 0.62%
 Time  (median):     443.718 ms               ┊ GC (median):    0.91%
 Time  (mean ± σ):   452.423 ms ± 135.848 ms  ┊ GC (mean ± σ):  3.30% ± 7.21%

  ▁                 ▁      ▁▁ ▁   ▁  ▁     ▁▁  ▁              █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁██▁█▁▁▁█▁▁█▁▁▁▁▁██▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  175 ms           Histogram: frequency by time          661 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 30264.

In [13]:
@btime main(show=false)

  282.275 ms (48580 allocations: 104.12 MiB)


In [16]:
display(@benchmark main(show=false))

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  154.499 ms … 805.494 ms  ┊ GC (min … max): 0.00% … 0.53%
 Time  (median):     489.060 ms               ┊ GC (median):    0.76%
 Time  (mean ± σ):   463.520 ms ± 218.922 ms  ┊ GC (mean ± σ):  3.40% ± 6.51%

  █             ▁ ▁      ▁       ▁    ▁▁▁                   ▁ ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█ ▁
  154 ms           Histogram: frequency by time          805 ms <

 Memory estimate: 3.53 MiB, allocs estimate: 30242.